# 4.1 - Procesos asíncronos


![async](images/async.png)




**[Documentación](https://docs.python.org/3/library/asyncio.html)**


**asyncio** es una biblioteca para escribir código [concurrente](https://es.wikipedia.org/wiki/Concurrencia_(inform%C3%A1tica)) utilizando la sintaxis async/await. Se utiliza como base en múltiples frameworks asíncronos de Python y provee un alto rendimiento en redes y servidores web, bibliotecas de conexión de base de datos, colas de tareas distribuidas, etc.

Suele encajar perfectamente para operaciones con límite de E/S y código de red estructurado de alto nivel. Además provee un conjunto de APIs de alto nivel para:

+ ejecutar corutinas de Python de manera concurrente y tener control total sobre su ejecución

+ realizar redes E/S y comunicación entre procesos(IPC)

+ controlar subprocesos

+ distribuir tareas a través de colas

+ sincronizar código concurrente

Adicionalmente, existen APIs de bajo nivel para desarrolladores de bibliotecas y frameworks para:

+ crear y administrar bucles de eventos, los cuales proveen APIs asíncronas para redes, ejecutando subprocesos, gestionando señales del sistema operativo, etc..

+ implementar protocolos eficientes utilizando transportes

+ bibliotecas puente basadas en retrollamadas y código con sintaxis async/wait

$$$$

Nosotros nos enfocaremos en el uso de bucles de eventos para la extracción de datos de la web.


### Ejemplo ESPN

Volvamos al ejemplo de scrapeo de la págine de ESPN. Vamos a realizar múltiples requests para obtener los datos de todos los equipos.


https://www.espn.com/soccer/competitions

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select   # seleccion de un dropdown


import time

import pandas as pd

import warnings
warnings.filterwarnings('ignore')


import asyncio

from tqdm.notebook import tqdm


In [ ]:
PATH='driver/chromedriver'

url = 'https://www.espn.com/soccer/competitions'

In [ ]:
driver = webdriver.Chrome(PATH)

driver.get(url)

In [ ]:
# cookies

aceptar = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')

aceptar.click()

In [ ]:
# seleccion equipos laliga

equipos = driver.find_element(By.XPATH, '//*[@id="fittPageContainer"]/div[3]/div/div/div/div[2]/div[1]/div/div[5]/div/section/div/div/span[2]/a')


equipos.click()

In [ ]:
caja = driver.find_element(By.CLASS_NAME, 'layout.is-9-3')

caja

In [ ]:
#stats de equipos

stats = caja.find_elements(By.CSS_SELECTOR, 'a.AnchorLink')

stats[10].get_attribute('href')

In [ ]:
len(stats)

In [ ]:
[e.get_attribute('href') for e in stats][:10]

In [ ]:
team_stats = []

for e in stats:
    
    try:
        
        link = e.get_attribute('href')
        
        if 'soccer/team/stats' in link:
            team_stats.append(link)
        else:
            continue
    except:
        continue
        
driver.quit()


team_stats[:10]

In [ ]:
len(team_stats)

**Extracción asincrónica**

In [ ]:
help(asyncio.get_event_loop().run_in_executor)

In [ ]:
def asincrono(funcion):
    
    def eventos(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, funcion, *args, **kwargs)
    
    return eventos

In [ ]:
url = 'https://www.espn.com/soccer/team/stats/_/id/3747/granada'

In [ ]:
DATOS=[]

CABECERAS=[]

In [ ]:
# paso 1

# inicia un driver
driver = webdriver.Chrome(PATH)
driver.get(url)

In [ ]:
# paso 2
# cookies
try:
    aceptar = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')

    aceptar.click()
    
except:
    print('Ya')

In [ ]:
# paso 3
dropdown = driver.find_element(By.XPATH, '//*[@id="fittPageContainer"]/div[2]/div[5]/div/div/section/div/div[4]/select[1]')
select = Select(dropdown)

try:
    select.select_by_visible_text('2022-23')
except:
    select.select_by_visible_text('2023-24')


In [ ]:
# paso 4
# disciplina
dis=driver.find_element(By.XPATH,'//*[@id="fittPageContainer"]/div[2]/div[5]/div/div[1]/section/div/div[2]/nav/ul/li[2]/a')
dis.click()

In [ ]:
# paso 5

tabla=driver.find_element(By.TAG_NAME,'tbody')

filas=tabla.find_elements(By.TAG_NAME, 'tr')


data=[]
    
for f in filas:

    elementos=f.find_elements(By.TAG_NAME, 'td') 

    tmp=[]

    for e in elementos:

        tmp.append(e.text)

    tmp.append(url.split('/')[-1])  # añade el nombre del equipo

    data.append(tmp)
    
    
        
cabeceras=driver.find_element(By.TAG_NAME, 'thead')

cabeceras=[c.text for c in cabeceras.find_elements(By.TAG_NAME, 'th')]+['TEAM']


DATOS+=data

CABECERAS=cabeceras


df = pd.DataFrame(DATOS, columns=CABECERAS)

df

In [ ]:
driver.quit()

In [ ]:
DATOS = []

CABECERAS = []

In [ ]:

def extraer(url):
    
    global DATOS, CABECERAS, PATH
    
    # paso 1
    # inicia un driver
    driver = webdriver.Chrome(PATH)
    driver.get(url)
    
    time.sleep(2)
    
    
    # paso 2
    # cookies
    try:
        aceptar = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')

        aceptar.click()

    except:
        print('Ya')
        
    time.sleep(2)
        
    
    # paso 3
    dropdown = driver.find_element(By.XPATH, '//*[@id="fittPageContainer"]/div[2]/div[5]/div/div/section/div/div[4]/select[1]')
    select = Select(dropdown)

    try:
        select.select_by_visible_text('2022-23')
    except:
        select.select_by_visible_text('2023-24')
        
        
    time.sleep(2)
        
        
    # paso 4
    # disciplina
    dis=driver.find_element(By.XPATH,'//*[@id="fittPageContainer"]/div[2]/div[5]/div/div[1]/section/div/div[2]/nav/ul/li[2]/a')
    dis.click()
    
    time.sleep(2)
    
    
    # paso 5
    tabla=driver.find_element(By.TAG_NAME,'tbody')

    filas=tabla.find_elements(By.TAG_NAME, 'tr')


    data=[]

    for f in filas:

        elementos=f.find_elements(By.TAG_NAME, 'td') 

        tmp=[]

        for e in elementos:

            tmp.append(e.text)

        tmp.append(url.split('/')[-1])  # añade el nombre del equipo

        data.append(tmp)



    cabeceras=driver.find_element(By.TAG_NAME, 'thead')

    cabeceras=[c.text for c in cabeceras.find_elements(By.TAG_NAME, 'th')]+['TEAM']


    DATOS+=data

    CABECERAS=cabeceras

In [ ]:
extraer(url)

In [ ]:
%%time

for url in tqdm(team_stats[:10]):
    
    res=extraer(url)
    print(res)

In [ ]:
df=pd.DataFrame(DATOS, columns=CABECERAS)

df.shape

In [ ]:
df.TEAM.unique()